# Preciting IMDB Scores
## Build a regression model to predict the IMDb score of a TV show or movie based on features such as release year, runtime, genres, and production countries.

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import json
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from pathlib import Path
import pandas as pd
from pathlib import Path
import pandas as pd
import ast

In [26]:


# Define the path to the directory containing the CSV files
data_dir = Path("C:/Users/Qazi Fabia Hoq/OneDrive/Documents/GitHub/netflix-rating-project/resources")

# Read the showData.csv file into a DataFrame
show_data_path = data_dir / "showData.csv"
show_data = pd.read_csv(show_data_path)

# Display the first few rows of the DataFrame to verify that it was read correctly
show_data.head()


,Unnamed: 0,id,title,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_score
0,0,ts300399,Five Came Back: The Reference Films,1945,TV-MA,51,['documentation'],['US'],1.0,NaN
1,1,ts22164,Monty Python's Flying Circus,1969,TV-14,30,"['comedy', 'european']",['GB'],4.0,8.8
2,2,ts45948,Monty Python's Fliegender Zirkus,1972,TV-MA,43,['comedy'],[],1.0,8.1
3,3,ts20681,Seinfeld,1989,TV-PG,24,['comedy'],['US'],9.0,8.9
4,4,ts22082,Knight Rider,1982,TV-PG,51,"['scifi', 'action', 'crime', 'drama']",['US'],4.0,6.9


In [27]:
# Read the movieData.csv file into a DataFrame
movie_data_path = data_dir / "movieData.csv"
movie_data = pd.read_csv(movie_data_path)

# Display the first few rows of the DataFrame to verify that it was read correctly
movie_data.head()


,Unnamed: 0,id,title,release_year,age_certification,runtime,genres,production_countries,imdb_score
0,0,tm84618,Taxi Driver,1976,R,114,"['drama', 'crime']",['US'],8.2
1,1,tm154986,Deliverance,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],7.7
2,2,tm127384,Monty Python and the Holy Grail,1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],8.2
3,3,tm120801,The Dirty Dozen,1967,NaN,150,"['war', 'action']","['GB', 'US']",7.7
4,4,tm70993,Life of Brian,1979,R,94,['comedy'],['GB'],8.0


# Data Preprocessing

In [28]:
combined_data = pd.concat([show_data, movie_data], ignore_index=True)


In [29]:
combined_data.to_csv('C:/Users/Qazi Fabia Hoq/OneDrive/Documents/GitHub/netflix-rating-project/resources/combinedmovieshow_data.csv', index=False)


In [30]:
combined_data = pd.read_csv('C:/Users/Qazi Fabia Hoq/OneDrive/Documents/GitHub/netflix-rating-project/resources/combinedmovieshow_data.csv')


In [31]:
combined_data.head()


,Unnamed: 0,id,title,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_score
0,0,ts300399,Five Came Back: The Reference Films,1945,TV-MA,51,['documentation'],['US'],1.0,NaN
1,1,ts22164,Monty Python's Flying Circus,1969,TV-14,30,"['comedy', 'european']",['GB'],4.0,8.8
2,2,ts45948,Monty Python's Fliegender Zirkus,1972,TV-MA,43,['comedy'],[],1.0,8.1
3,3,ts20681,Seinfeld,1989,TV-PG,24,['comedy'],['US'],9.0,8.9
4,4,ts22082,Knight Rider,1982,TV-PG,51,"['scifi', 'action', 'crime', 'drama']",['US'],4.0,6.9


In [32]:
# Initialize an empty set to store unique genres
unique_genres = set()

# Iterate over the 'genres' column and add each genre to the set
combined_data['genres'].apply(lambda x: unique_genres.update([genre.strip().strip(" '[]") for genre in x.split(',')]))

# Print the unique genres
print(unique_genres)


{'', 'romance', 'animation', 'european', 'thriller', 'comedy', 'crime', 'war', 'history', 'documentation', 'horror', 'western', 'family', 'action', 'scifi', 'music', 'reality', 'drama', 'fantasy', 'sport'}


In [33]:
# Clean up the 'genres' column
combined_data['genres'] = combined_data['genres'].apply(lambda x: [genre.strip().strip(" '[]") for genre in x.split(',')])

# Extract unique genres
unique_genres = set()
combined_data['genres'].apply(lambda x: unique_genres.update(x))

# Encode genres
for genre in unique_genres:
    combined_data[genre] = combined_data['genres'].apply(lambda x: 1 if genre in x else 0)

# Print unique genres
print(unique_genres)


{'', 'romance', 'animation', 'european', 'thriller', 'comedy', 'crime', 'war', 'history', 'documentation', 'horror', 'western', 'family', 'action', 'scifi', 'music', 'reality', 'drama', 'fantasy', 'sport'}


In [34]:
# Remove rows with missing values in the target variable
combined_data = combined_data.dropna(subset=['imdb_score'])

# Feature Selection

In [35]:
X = combined_data[['release_year', 'runtime', 'western', 'fantasy', 'history', 'romance', 'crime', 'european', 'comedy', 'action', 'sport', 'thriller', 'music', 'animation', 'war', 'reality', 'horror', 'scifi', 'family', 'drama', 'documentation']]
y = combined_data['imdb_score']


# Split The Data

In [36]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shape of the training and test sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (4294, 21)
X_test shape: (1074, 21)
y_train shape: (4294,)
y_test shape: (1074,)


# Model Selection

In [37]:

# Initialize the models
models = {
    'Linear Regression': LinearRegression()
}


# Train The Model

In [38]:
for name, model in models.items():
    print(f"Training {name}...")
    # Train the model
    model.fit(X_train, y_train)
    print(f"{name} trained.")


Training Linear Regression...
Linear Regression trained.


In [39]:
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)


Coefficients: [-0.02773994 -0.00526004 -0.11579916  0.18058725  0.41580719 -0.12664494
  0.14586746 -0.01860548 -0.00803708 -0.20385965  0.03635262 -0.07652127
  0.00420428  0.44966579  0.22323952 -0.09041945 -0.48724668 -0.00947496
 -0.43177198  0.51540985  0.72176608]
Intercept: 62.5086948263234


# Make Prediction

In [40]:
# Make predictions
y_pred = model.predict(X_test)

# Print predictions
print("Predictions:", y_pred)

Predictions: [6.42671293 6.4073848  6.61414361 ... 6.68420992 6.50555588 6.14995543]


# Evaluate The Performance

In [41]:
# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

# Print the metrics
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Absolute Error: 0.823485196119407
Mean Squared Error: 1.1243918647340323
R-squared: 0.15549410044771972
